In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import scipy
import copy
import matplotlib.gridspec as gridspec
import pybaselines.polynomial
from scipy import signal
%matplotlib inline

In [ ]:
# Import warnings; and specify them all to be ignored.
import warnings
warnings.filterwarnings('ignore')
plt.rcdefaults()

In [ ]:
def start_row(path):
    myFile = open(path)
    text = myFile.readline()
    row = 0
    while 'Potential/V' not in text:
        row = row + 1
        text = myFile.readline()
    myFile.close()
    return row

In [ ]:
def peak_range(signal_df, v_peak, width):
    temp_df = copy.deepcopy(signal_df)
    b, a = signal.butter(2, 0.1, fs=5)
    temp_df[' Diff(i/A)'] = signal.filtfilt(b, a, temp_df[' Diff(i/A)'])
    pi = temp_df[ temp_df['Potential/V'] == v_peak ].index[0]
    (imax, jmax) = (0, 0)
    ph1 = abs(temp_df[ temp_df['Potential/V'] == v_peak ][' Diff(i/A)'].iloc[0])
    phmax = 0
    v_1 = 0
    v_2 = 0
    sl_max = 0
    in_max = 0
    
    if (pi+width > len(temp_df[' Diff(i/A)'])):
        width = len(temp_df[' Diff(i/A)'])-pi
    
    if (pi < width):
        width = pi
        
    for i in range(1,width):
        for j in range(1,width):
            ytemp = [temp_df[' Diff(i/A)'][pi - j], temp_df[' Diff(i/A)'][pi + i]]
            xtemp = [temp_df['Potential/V'][pi - j], temp_df['Potential/V'][pi + i]]
            slope, intercept, r, p, se = scipy.stats.linregress(xtemp, ytemp)
            ph2 = abs( slope*temp_df['Potential/V'][pi]+intercept )
            ph = ph1 - ph2
            # print(ph)
            if ph > phmax:
                sl_max = slope
                in_max = intercept
                phmax = ph
                (imax, jmax) = (i, j)
                v_1 = temp_df['Potential/V'][pi - jmax]
                v_2 = temp_df['Potential/V'][pi + imax]
    temp_df['v_peak'] = temp_df[ (temp_df['Potential/V'] > v_1) & (temp_df['Potential/V'] < v_2) ]['Potential/V']
    temp_df['i_peak'] = temp_df[ (temp_df['Potential/V'] > v_1) & (temp_df['Potential/V'] < v_2) ][' Diff(i/A)']
    temp_df['i_peak'] = abs( temp_df['i_peak'] - (temp_df['v_peak']*sl_max + in_max) )
    return temp_df, v_1, v_2

In [ ]:
def Fitting(signal_df, cut_begin, cut_end, peak_start, peak_end, p_order, mode: str):
    temp_df = copy.deepcopy(signal_df)
    
    if cut_begin != 0:
        temp_df = temp_df[ temp_df['Potential/V'] > cut_begin ]

    if cut_end != 0:
        temp_df = temp_df[ temp_df['Potential/V'] < cut_end ]
    
    if mode == 'DPV':
        temp_df[' Current/A'] = abs(temp_df[' Current/A'])
        signal_df_masked = temp_df[ (temp_df['Potential/V'] < peak_start) | (temp_df['Potential/V'] > peak_end) ]
        _, params = pybaselines.polynomial.poly(signal_df_masked[' Current/A'], signal_df_masked['Potential/V'], poly_order=p_order, return_coef=True)
        signal_bline = np.polynomial.Polynomial(params['coef'])(temp_df['Potential/V'])
        temp_df['BackgroundI'] = signal_bline
        temp_df['SubtractedI'] = temp_df[' Current/A']-signal_bline
        
    elif mode == 'SWV':
        temp_df[' Diff(i/A)'] = abs(temp_df[' Diff(i/A)'])
        signal_df_masked = temp_df[ (temp_df['Potential/V'] < peak_start) | (temp_df['Potential/V'] > peak_end) ]
        _, params = pybaselines.polynomial.poly(signal_df_masked[' Diff(i/A)'], signal_df_masked['Potential/V'], poly_order=p_order, return_coef=True)
        signal_bline = np.polynomial.Polynomial(params['coef'])(temp_df['Potential/V'])
        temp_df['BackgroundI'] = signal_bline
        temp_df['SubtractedI'] = temp_df[' Diff(i/A)']-signal_bline
    
    return temp_df

In [ ]:
E22 = { '0':['E22/0 nm.txt'],
       '10':['E22/10 nm.txt'],
       '50':['E22/50 nm.txt'],
       '100':['E22/110 nm.txt'],
       '200':['E22/210 nm.txt'],
       '400':['E22/410 nm.txt'],
       '500':['E22/510 nM.txt'],
       '700':['E22/710 nM.txt'],
       '1000':['E22/1010 nM.txt']}

In [ ]:
for dict in [E22]:
    for key in dict.keys():
        for i in range(len(dict[key])):
            temp_df = pd.read_csv(dict[key][i], skiprows=start_row(dict[key][i]))
            temp_df, v_1, v_2 = peak_range(temp_df, 0.340, 60)
            if v_1 == 0:
                v_1 = 0.28
                v_2 = 0.48
            temp_df = Fitting(temp_df, cut_begin=0.25, cut_end=0.52, peak_start=v_1, peak_end=v_2, p_order=3, mode='SWV')
            dict[key][i] = temp_df

In [ ]:
df_peaks = pd.DataFrame(columns=[])

for (dict,elec) in [(E22,'E22')]:
    for key in dict.keys():
        temp_list = [elec,key]
        for df in dict[key]:
            temp_list.append(1e9*df[ (df['Potential/V'] > 0.3) & (df['Potential/V'] < 0.45) ]['SubtractedI'].max())
        temp_df = pd.DataFrame(temp_list)
        df_peaks = pd.concat([df_peaks, temp_df.T], ignore_index=True, axis=0)

df_peaks.set_index(0, inplace=True)
df_peaks.to_excel('Peak_values.xlsx')

In [ ]:
df_cal = pd.read_excel('Peak_values_e.xlsx', index_col='conc')
x_cal = df_cal.index.astype('float')
y_cal = df_cal['AVG'].astype('float')

slope_cal, intercept_cal, r_value, p_value, std_err = scipy.stats.linregress(x_cal, y_cal)
text_cal = '\n'.join((
    r'$y=%.2fx+%.2f$' % (slope_cal, intercept_cal),
    r'$R^2=%.3f$' % (r_value**2)))